# fastkafka-gen

> The FastKafka-gen library uses advanced AI to generate FastKafka code from user descriptions, speeding up FastKafka app development.

This file will become your README and also the index of your documentation.

## Install

```sh
pip install faststream_gen
```

```sh
faststream_gen --help
```

In [3]:
# | echo: false
! faststream_gen --help

                                                                                
 Usage: faststream_gen [OPTIONS] [DESCRIPTION]                                  
                                                                                
 Effortlessly generate FastStream application code and integration tests from   
 the app description.                                                           
                                                                                
╭─ Arguments ──────────────────────────────────────────────────────────────────╮
│   description      [DESCRIPTION]  Summarize your FastStream app in a few     │
│                                   sentences!                                 │
│                                                                              │
│                                   Include details about messages, topics,    │
│                                   servers, and a brief overview of the       │
│                           

## Example

```sh
faststream_gen  "Write a faststream application with with one consumer function and two producer functions. The consumer function should receive the a message posted on 'new_joinee' topic. The message should contain 'employee_name', 'age', 'location' and 'experience' attributes. After consuming the consumer function should send the details to the 'project_team' and 'admin_team' topics." 
✨  Generating a new FastStream application!
 ✔ Application description validated 
 ✔ FastStream app skeleton generated and saved at: ./faststream-gen/application_skeleton.py 
 ✔ FastStream app generated and saved at: ./faststream-gen/application.py 
 ✔ Tests are generated and saved at: ./faststream-gen/test.py 
 Tokens used: 18572
 Total Cost (USD): $0.05635
✨  All files were successfully generated!
```

**Note**: If you have longer application description, it is easier to save description to file and use:
```sh
faststream_gen  -i path_to_app_description/description.txt
```

`faststream_gen` CLI will create `faststream-gen` folder and save generated FastStream `application.py` inside the folder. We know that no one likes code full of errors, that's why we generate and run tests along with the generated application! Next to `application.py` you will find the `test.py` script for testing your FastStream application!

`faststream-gen/application.py`:

In [16]:
# | echo: false

from IPython.display import Code

Code(filename='../docs_src/app_index_example/application.py', language='python')

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Employee(BaseModel):
    employee_name: str = Field(
        ..., examples=["John Doe"], description="Employee name"
    )
    age: int = Field(
        ..., examples=[30], description="Age"
    )
    location: str = Field(
        ..., examples=["New York"], description="Location"
    )
    experience: int = Field(
        ..., examples=[5], description="Experience in years"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.subscriber("new_joinee")
@broker.publisher("project_team")
@broker.publisher("admin_team")
async def on_new_joinee(msg: Employee, logger: Logger):
    logger.info(msg)
    return msg

In [ ]:
%load ../docs_src/app_index_example/application.py
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Employee(BaseModel):
    employee_name: str = Field(
        ..., examples=["John Doe"], description="Employee name"
    )
    age: int = Field(
        ..., examples=[30], description="Age"
    )
    location: str = Field(
        ..., examples=["New York"], description="Location"
    )
    experience: int = Field(
        ..., examples=[5], description="Experience in years"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.subscriber("new_joinee")
@broker.publisher("project_team")
@broker.publisher("admin_team")
async def on_new_joinee(msg: Employee, logger: Logger):
    logger.info(msg)
    return msg

`faststream-gen/test.py`:

In [18]:
# | echo: false

from IPython.display import Code

Code(filename='../docs_src/app_index_example/application.py', language='python')

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Employee(BaseModel):
    employee_name: str = Field(
        ..., examples=["John Doe"], description="Employee name"
    )
    age: int = Field(
        ..., examples=[30], description="Age"
    )
    location: str = Field(
        ..., examples=["New York"], description="Location"
    )
    experience: int = Field(
        ..., examples=[5], description="Experience in years"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.subscriber("new_joinee")
@broker.publisher("project_team")
@broker.publisher("admin_team")
async def on_new_joinee(msg: Employee, logger: Logger):
    logger.info(msg)
    return msg

## How to use

Fill me in please! Don't forget code examples:

In [2]:
1+1

2